<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [97]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [153]:
portfolio = pd.read_json('data/portfolio.json', orient='records', lines=True)
profile = pd.read_json('data/profile.json', orient='records', lines=True)
transcript = pd.read_json('data/transcript.json', orient='records', lines=True)

In [154]:
portfolio.head()

,channels,difficulty,duration,id,offer_type,reward
0,"[email, mobile, social]",10,7,ae264e3637204a6fb9bb56bc8210ddfd,bogo,10
1,"[web, email, mobile, social]",10,5,4d5c57ea9a6940dd891ad53e9dbe8da0,bogo,10
2,"[web, email, mobile]",0,4,3f207df678b143eea3cee63160fa8bed,informational,0
3,"[web, email, mobile]",5,7,9b98b8c7a33c4b65b9aebfe6a799e6d9,bogo,5
4,"[web, email]",20,10,0b1e1539f2cc45b7b9fa7c272da2e1d7,discount,5


In [155]:
transcript.head()

,event,person,time,value
0,offer received,78afa995795e4d85b5d9ceeca43f5fef,0,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'}
1,offer received,a03223e636434f42ac4c3df47e8bac43,0,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'}
2,offer received,e2127556f4f64592b11af22de27a7932,0,{'offer id': '2906b810c7d4411798c6938adc9daaa5'}
3,offer received,8ec6ce2a7e7949b1bf142def7d0e0586,0,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'}
4,offer received,68617ca6246f4fbc85e91a2a49552598,0,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'}


In [156]:
profile.head()

,age,became_member_on,gender,id,income
0,118,20170212,None,68be06ca386d4c31939f3a4f0e3dd783,NaN
1,55,20170715,F,0610b486422d4921ae7d2bf64640c50b,112000.0
2,118,20180712,None,38fe809add3b4fcf9315a9694bb96ff5,NaN
3,75,20170509,F,78afa995795e4d85b5d9ceeca43f5fef,100000.0
4,118,20170804,None,a03223e636434f42ac4c3df47e8bac43,NaN


In [157]:
df_1 = pd.merge(left=profile, right=transcript, left_on='id', right_on='person')

In [158]:
df_1 = df_1.drop('person', axis=1)

In [159]:
df_1.head()

,age,became_member_on,gender,id,income,event,time,value
0,118,20170212,None,68be06ca386d4c31939f3a4f0e3dd783,NaN,offer received,168,{'offer id': '2906b810c7d4411798c6938adc9daaa5'}
1,118,20170212,None,68be06ca386d4c31939f3a4f0e3dd783,NaN,offer viewed,216,{'offer id': '2906b810c7d4411798c6938adc9daaa5'}
2,118,20170212,None,68be06ca386d4c31939f3a4f0e3dd783,NaN,offer received,336,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'}
3,118,20170212,None,68be06ca386d4c31939f3a4f0e3dd783,NaN,offer viewed,348,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'}
4,118,20170212,None,68be06ca386d4c31939f3a4f0e3dd783,NaN,transaction,360,{'amount': 0.35000000000000003}


In [160]:
def parse_value_keys(row, value_type):
    if value_type in row.keys():
        label = row[value_type]
        return label
    else: pass

In [161]:
df_1['reward_amount'] = df_1.value.apply(lambda x: parse_value_keys(x, 'reward'))
df_1['transaction_amount'] = df_1.value.apply(lambda x: parse_value_keys(x, 'amount'))
df_1['offer_id'] = df_1.value.apply(lambda x: parse_value_keys(x, 'offer id'))

In [162]:
df_2 = pd.merge(left=df_1, right=portfolio, left_on='offer_id', right_on='id', how='outer',suffixes=('_customer','_offer'))

In [163]:
df_complete = df_2.drop(['value','id_offer'], axis=1)

In [164]:
df_complete.head()

,age,became_member_on,gender,id_customer,income,event,time,reward_amount,transaction_amount,offer_id,channels,difficulty,duration,offer_type,reward
0,118,20170212,None,68be06ca386d4c31939f3a4f0e3dd783,NaN,offer received,168,NaN,NaN,2906b810c7d4411798c6938adc9daaa5,"[web, email, mobile]",10.0,7.0,discount,2.0
1,118,20170212,None,68be06ca386d4c31939f3a4f0e3dd783,NaN,offer viewed,216,NaN,NaN,2906b810c7d4411798c6938adc9daaa5,"[web, email, mobile]",10.0,7.0,discount,2.0
2,68,20180426,M,e2127556f4f64592b11af22de27a7932,70000.0,offer received,0,NaN,NaN,2906b810c7d4411798c6938adc9daaa5,"[web, email, mobile]",10.0,7.0,discount,2.0
3,68,20180426,M,e2127556f4f64592b11af22de27a7932,70000.0,offer viewed,18,NaN,NaN,2906b810c7d4411798c6938adc9daaa5,"[web, email, mobile]",10.0,7.0,discount,2.0
4,118,20170925,None,8ec6ce2a7e7949b1bf142def7d0e0586,NaN,offer received,408,NaN,NaN,2906b810c7d4411798c6938adc9daaa5,"[web, email, mobile]",10.0,7.0,discount,2.0


In [165]:
df_complete['channels'] = df_complete['channels'].str.len()

In [166]:
df_complete.isnull().sum()

age                        0
became_member_on           0
gender                 33772
id_customer                0
income                 33772
event                      0
time                       0
reward_amount         272955
transaction_amount    167581
offer_id              172532
channels              172532
difficulty            172532
duration              172532
offer_type            172532
reward                172532
dtype: int64

In [167]:
f'percent of gender column missing: {df_complete.isnull().sum()[2]/len(df_complete):.2%}'

'percent of gender column missing: 11.02%'

`channels` column convert to length. All channels have `[web, email]` which is now 2. `[web, email, mobile]` is three and `[web, email, mobile, social]` is now 4. 

The offer types are also numeric now `bogo` = 1, `discount` = 2, and `informational` = 3

Trying a new idea: offer recieved and offer viewed, while important for demographics, is probably not all that important in so much as they both may not convert to a sale or offer completion. As such, I'll group them together and try to predict completion, transaction, or none.

TARGET MODEL:
`offer viewed` and `offer received` = 0, `transaction` = 1, `offer completed` = 2

In [168]:
def parse_offers(row):
    if row == 'bogo':
        label = 1
    elif row == 'discount':
        label = 2
    else: label = 3
    return label

In [169]:
df_complete['offer_type'] = df_complete.offer_type.apply(lambda x: parse_offers(x))

In [170]:
events = {'offer received':0,'offer viewed':0,'offer completed':2,'transaction':1}
genders = {'M':1,'F':2, 'O':3, None:0}

df_complete.event = [events[item] for item in df_complete.event]
df_complete.gender = [genders[item] for item in df_complete.gender]

In [171]:
df_complete.income.fillna(value=income_med, inplace=True)

In [172]:
df_complete.id_customer = [ hash(uid) for uid in df_complete.id_customer]
df_complete.offer_id = [hash(oid) for oid in df_complete.offer_id]

In [173]:
df_complete.fillna(value=0, inplace=True)

In [174]:
df_complete.head()

,age,became_member_on,gender,id_customer,income,event,time,reward_amount,transaction_amount,offer_id,channels,difficulty,duration,offer_type,reward
0,118,20170212,0,-3007744034810710241,62000.0,0,168,0.0,0.0,8603045624753757428,3.0,10.0,7.0,2,2.0
1,118,20170212,0,-3007744034810710241,62000.0,0,216,0.0,0.0,8603045624753757428,3.0,10.0,7.0,2,2.0
2,68,20180426,1,-2450949750221503192,70000.0,0,0,0.0,0.0,8603045624753757428,3.0,10.0,7.0,2,2.0
3,68,20180426,1,-2450949750221503192,70000.0,0,18,0.0,0.0,8603045624753757428,3.0,10.0,7.0,2,2.0
4,118,20170925,0,5506084018727061253,62000.0,0,408,0.0,0.0,8603045624753757428,3.0,10.0,7.0,2,2.0


In [175]:
import joblib

In [190]:
joblib.dump(df_complete, 'data/dataframe')

['data/dataframe']

In [4]:
#df_complete = joblib.load('data/dataframe')

Let's see if we can do a prediction for users who are likely to acheive any of the four labeled **events**. We can tag this outcome with probabilities to understand who is most likely to achieve each event type. 

We can then possible work backwards to understand the profile of a customer who is likely to acheive a given offer - or which offers typically give us specific **event** outcomes. 

In [176]:
from ml_test_tools import test_model, rank_top_performers

In [ ]:
# !conda install -y -c conda-forge xgboost

In [177]:
import xgboost as xgb

In [178]:
models = {}

In [179]:
from sklearn.model_selection import train_test_split

In [180]:
X = df_complete.drop('event', axis=1)
y = df_complete.event

In [181]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=9450)

In [182]:
training_set = pd.concat([X_train, y_train],axis=1)

In [183]:
import random
N_TRIALS = 3
for i in range(N_TRIALS):
    learning_rate_factor = random.randint(1,25)
    xgb_rs = xgb.XGBClassifier(
        learning_rate=0.25 / learning_rate_factor,
        n_estimators=random.randint(20, 400),
        max_depth=random.randint(2, 10),
        min_child_weight=random.randint(1, 10),
        gamma=random.randint(0, 50)/5,
        subsample=1-(random.randint(1, 60)/100),
        colsample_bytree=1-(random.randint(1, 60)/100),
        random_state=9450, 
        objective='multi:softmax'
    )
    model = test_model(training_set, xgb_rs, cv_folds=5, model_id=i)
    models.update({i:model})

In [184]:
rank_top_performers(models, metric='accuracy')

[{'model_id': 0,
  'model': XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
                colsample_bytree=0.85, gamma=9.0,
                learning_rate=0.019230769230769232, max_delta_step=0, max_depth=3,
                min_child_weight=5, missing=None, n_estimators=177, n_jobs=1,
                nthread=None, objective='multi:softprob', random_state=9450,
                reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
                silent=True, subsample=0.9),
  'f1_score': {'mean': 1.0, 'sd': 0.0},
  'recall': {'mean': 1.0, 'sd': 0.0},
  'precision': {'mean': 1.0, 'sd': 0.0},
  'specificity': {'mean': 1.0, 'sd': 0.0},
  'balanced_accuracy': {'mean': 1.0, 'sd': 0.0},
  'accuracy': {'mean': 1.0, 'sd': 0.0}},
 {'model_id': 1,
  'model': XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
                colsample_bytree=0.9299999999999999, gamma=0.2,
                learning_rate=0.0625, max_delta_step=0, max_depth=8,
              

In [185]:
top_model = models[0]['model']
top_model

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bytree=0.85, gamma=9.0,
              learning_rate=0.019230769230769232, max_delta_step=0, max_depth=3,
              min_child_weight=5, missing=None, n_estimators=177, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=9450,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=True, subsample=0.9)

In [191]:
joblib.dump(top_model, 'data/xgb_model_1')

['data/xgb_model_1']

In [186]:
val_preds = top_model.predict(X_test.values)

In [187]:
from sklearn.metrics import confusion_matrix, classification_report,accuracy_score

In [188]:
con_mat = confusion_matrix(y_true=y_test, y_pred=val_preds)
cls_rpt = classification_report(y_test, val_preds)
acc = accuracy_score(y_test, val_preds)

In [189]:
print(con_mat, cls_rpt, f'accuracy score: {acc:.2%}', sep='\n\n')

[[13235     0     0]
 [    0 14003     0]
 [    0     0  3416]]

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     13235
           1       1.00      1.00      1.00     14003
           2       1.00      1.00      1.00      3416

    accuracy                           1.00     30654
   macro avg       1.00      1.00      1.00     30654
weighted avg       1.00      1.00      1.00     30654


accuracy score: 100.00%


Events are numeric as such: 
    
`offer viewed` and `offer received` = 0, `offer completed` = 2, `transaction` = 1

This is a great result for predicting something like a transaction or offer completion. If the goal is to find who will buy without offer and who will buy with an offer, then this is pretty good. Possible future work would be to determine the optimal difficulty level needed to complete an offer.

I thought there would be some drop off in the accuracy as a result of the validation set, but I guess not. Seems too good. 

In [201]:
random_classes = np.random.randint(0,3, size=len(y_test))

In [202]:
con_mat_ran = confusion_matrix(y_true=y_test, y_pred=random_classes)
cls_rpt_ran = classification_report(y_test, random_classes)
acc_ran = accuracy_score(y_test, random_classes)

In [204]:
print(con_mat_ran, cls_rpt_ran, f'accuracy score for random class distribution: {acc_ran:.2%}', sep='\n\n')

[[4407 4436 4392]
 [4733 4595 4675]
 [1140 1141 1135]]

              precision    recall  f1-score   support

           0       0.43      0.33      0.37     13235
           1       0.45      0.33      0.38     14003
           2       0.11      0.33      0.17      3416

    accuracy                           0.33     30654
   macro avg       0.33      0.33      0.31     30654
weighted avg       0.40      0.33      0.35     30654


accuracy score for random class distribution: 33.07%


So a random selection of classes, would be what we expect in terms of accuracy. The predictive power of completing offers is particularly low for a random distribution of offer completion classes. 